In [1]:
import numpy as np, pandas as pd
import glob
import os 
import json
import sys
import numpy
from math import log, log10
from gdpyc import GasMap, DustMap
from astropy.coordinates import SkyCoord

df = pd.read_csv('/Users/huiyang/Research/GitHub/MUWCLASS_4FGL/codes/demo/FGL_more/FGL_11152023_GammaCan.csv')

df = df[df['FGL'].str[-1]!='c']

df = df.sort_values(by=['flux_aper90_ave_b'], ascending=False).reset_index(drop=True)

# print(df.columns)

df[['FGL','X_name','Class','name','flux_aper90_ave_b']]

df['name_nospace'] = df['X_name'].str.replace(' ','')#apply(lambda x: x.replace(' ', ''))



df_alls = pd.read_csv('/Users/huiyang/Research/GitHub/MUWCLASS_4FGL/codes/Classifications/data/Individual_sources/FGL_11152023_GammaCan_alls.csv')

# light curve >50 counts
# spectrum > 100 counts

df_select3 = df[df['X_name'].isin(df_alls.loc[df_alls['src_cnts_aper90_b']>=50, 'name'].unique())].reset_index(drop=True)

df_select3 = df_select3[~df_select3['X_name'].isin(['2CXO J174457.8-290509', '2CXO J174507.0-290357','2CXO J174506.8-290537','2CXO J111459.1-611707','2CXO J150234.5+015205','2CXO J073717.0+653557'])].reset_index(drop=True)
# print(df_select3)

coords = SkyCoord(df_select3['ra'], df_select3['dec'], unit='deg')
df_select3['ebv'] = DustMap.ebv(coords, dustmap='SFD') * 0.86 # 0.86 is the correction described in Schlafly et al. 2010 and Schlafly & Finkbeiner 2011
df_select3['nH_from_AV'] = 2.21 * 3.1 * df_select3['ebv'] * 0.1
df_select3['nH']  = np.log10(GasMap.nh(coords, nhmap='LAB').value) #/ 1e22 # nH in unit of 1.e22 atoms /cm2

print(df_select3[['FGL','X_name','Class','CT','name','name_nospace','ra', 'dec','flux_aper90_ave_b','var_intra_prob',
       'var_inter_prob','nH']])

             FGL                 X_name    Class         CT  \
0   J0859.3-4342  2CXO J085927.0-434528       CV   2.451807   
1   J1725.1-3408  2CXO J172508.8-341112      YSO   0.335596   
2   J0859.2-4729  2CXO J085905.6-473041      YSO   4.286869   
3   J1742.5-2833  2CXO J174216.9-283707       CV   1.907402   
4   J1743.8-3143  2CXO J174347.2-314025  LM-STAR   4.563578   
..           ...                    ...      ...        ...   
56  J1510.9+0551  2CXO J151100.4+054921      AGN   3.911411   
57  J0859.3-4342  2CXO J085928.4-434628       NS   2.893635   
58  J1843.7-3227  2CXO J184332.8-322122      AGN   2.564143   
59  J1510.9+0551  2CXO J151051.3+054937      AGN  41.413040   
60  J1242.6+3236  2CXO J124235.1+323340      AGN  24.799075   

                       name          name_nospace          ra        dec  \
0   2CXO J085927.0-434528-1  2CXOJ085927.0-434528  134.862609 -43.757805   
1   2CXO J172508.8-341112-1  2CXOJ172508.8-341112  261.286915 -34.186814   
2   2CXO J08590

In [2]:
df_srcs = pd.DataFrame()


for index, df_src in df_select3.iterrows():# ['name_nospace'].values:
    
    src = df_src['name_nospace']
    src_short = src[4:]
    # if src != '2CXOJ174457.8-290509':
    #     continue
    #print(src)
    # if index >40:
    #     continue

    # df_src = df_select[df_select['name_nospace']==src]
    ra, dec, src_name = df_src['ra'], df_src['dec'], df_src['X_name']
    # print(src_name, ra, dec)
    
    src_stat = {'name': src_short, 'Class':df_src['Class'], 'CT':df_src['CT'], 'nH_LAB':df_src['nH'], 'var_inter':df_src['var_inter_prob'], 'var_intra':df_src['var_intra_prob']}
    # coords = SkyCoord(ra, dec, unit='deg')
    # ebv = DustMap.ebv(coords, dustmap='SFD') * 0.86 # 0.86 is the correction described in Schlafly et al. 2010 and Schlafly & Finkbeiner 2011
    # nH_from_AV = df_src['nH_from_AV']# 2.21 * 3.1 * ebv * 0.1
    # nH  = df_src['nH']# GasMap.nh(coords, nhmap='LAB').value / 1e22 # nH in unit of 1.e22 atoms /cm2
    #print(f'nH: {nH} 1.e22 atoms /cm2')

    # df_var = search_file.table.to_pandas()
    df_var = df_alls[(df_alls['name']==src_name) & (df_alls['src_cnts_aper90_b']>=50)].reset_index(drop=True)
    # print(df_var[['name','obsid','obi','region_id','src_cnts_aper90_b']])
    # print(df_var['powlaw_nh'][0]/100,nH_from_AV,nH)
    
    # '''
    df_var['obsid'] = df_var['obsid'].apply(lambda x: str(x).zfill(5))

    src_stat['nobs'] = len(df_var)
    src_stat['ncounts'] = df_var['src_cnts_aper90_b'].sum()
    src_stat['fb'] = df_src['flux_aper90_ave_b']
    # os.system(f'python customfitter_wstat_logpar.py -fn {src}_files.txt -md twomekal')
    # os.system(f'python customfitter_wstat_logpar.py -fn {src}_files.txt -md powerlawbb')

    dir_abs = f'/Users/huiyang/Research/GitHub/FGL_spectral_summary/bxa_results/{src}/'

    
    # os.system(f'python model_compare.py {dir_abs}powerlaw_out_/ {dir_abs}mekal_out_/ {dir_abs}bb_out_/  > {src}_model_compare.txt')

    prefixes = [f'{dir_abs}powerlaw_out_/',  f'{dir_abs}mekal_out_/', f'{dir_abs}bb_out_/']
    models = dict([(f, json.load(open(f + "info/results.json"))['logz']) for f in prefixes])

    best = max(models, key=models.__getitem__)
    Zbest = models[best]
    for m in models: models[m] -= Zbest
    Ztotal = log(sum(numpy.exp([Z for Z in list(models.values())])))
    limit = 30 # for example, Jeffreys scale for the Bayes factor

    # print()
    # print('Model comparison')
    # print('****************')
    # print()



    for m, md, md_f,p_n,p_l_n,p_u_n in zip(models, ['pl', 'mk', 'bb'], ['powerlaw','mekal','bb'], ['pl1.PhoIndex_median','src1.logkT_median','src1.logkT_median'], ['pl1.PhoIndex_errlo','src1.logkT_errlo','src1.logkT_errlo'], ['pl1.PhoIndex_errup','src1.logkT_errup','src1.logkT_errup']):
        df_src_sum = pd.read_csv(f'{dir_abs}{md_f}_out_/info/post_summary.csv')
        # print(df_src_sum.columns)
        src_stat[md],src_stat[md+'_l'],src_stat[md+'_u'] = df_src_sum.loc[0, p_n],df_src_sum.loc[0, p_l_n],df_src_sum.loc[0, p_u_n]
        Zrel = models[m]
        src_stat['log10Z_'+md] = Zrel / log(10)
        if Zrel == 0:
            src_stat['nH'],src_stat['nH_l'],src_stat['nH_u'] = df_src_sum.loc[0, 'src1.lognH_median'],df_src_sum.loc[0, 'src1.lognH_errlo'],df_src_sum.loc[0, 'src1.lognH_errup']
    df_sr = pd.DataFrame(data=src_stat, index=[0])
    df_srcs = pd.concat([df_srcs, df_sr], ignore_index=True, sort=False)
    # for md in ['powerlaw','mekal','bb']:

    print('  <tr>')
    print(f"   <td>{src_stat['name']}</td>")
    print(f"   <td>{src_stat['Class']}</td>")
    print(f"   <td>{src_stat['CT']:.2f}</td>")
    print(f"   <td>{src_stat['fb']:.2e}</td>")
    print(f"   <td>{src_stat['var_inter']:.2f}</td>")
    print(f'   <td><a class="image-link" href="./images/{src}_lc.png" target="_blank">{src_stat["var_intra"]:.2f}</a></td>')
    print(f"   <td>{src_stat['nobs']:.1f}</td>")
    print(f"   <td>{src_stat['ncounts']:.1f}</td>")
    print(f"   <td>{src_stat['pl']:.2f}<sup>+{src_stat['pl_u']-src_stat['pl']:.2f}</sup><sub style='position: relative; right: +2.3em;'>-{src_stat['pl']-src_stat['pl_l']:.2}</sub></td>")
    # print(f"   <td>{src_stat['log10Z_pl']:.2f}</td>")
    print(f'   <td><a class="image-link" href="./images/{src}_pl_spectrum.png" target="_blank">{src_stat["log10Z_pl"]:.2f}</a> (<a class="image-link" href="./images/{src}_pl_corner.pdf" target="_blank">covar</a>)</td>')
    print(f"   <td>{10**src_stat['mk']:.2f}<sup>+{10**src_stat['mk_u']-10**src_stat['mk']:.2f}</sup><sub style='position: relative; right: +2.3em;'>-{10**src_stat['mk']-10**src_stat['mk_l']:.2}</sub></td>")
    print(f'   <td><a class="image-link" href="./images/{src}_mk_spectrum.png" target="_blank">{src_stat["log10Z_mk"]:.2f}</a> (<a class="image-link" href="./images/{src}_mk_corner.pdf" target="_blank">covar</a>)</td>')
    # print(f"   <td>{src_stat['log10Z_mk']:.2f}</td>")
    print(f"   <td>{10**src_stat['bb']:.2f}<sup>+{10**src_stat['bb_u']-10**src_stat['bb']:.2f}</sup><sub style='position: relative; right: +2.3em;'>-{10**src_stat['bb']-10**src_stat['bb_l']:.2}</sub></td>")
    # print(f"   <td>{src_stat['log10Z_bb']:.2f}</td>")
    print(f'   <td><a class="image-link" href="./images/{src}_bb_spectrum.png" target="_blank">{src_stat["log10Z_bb"]:.2f}</a> (<a class="image-link" href="./images/{src}_bb_corner.pdf" target="_blank">covar</a>)</td>')
    print(f"   <td>{src_stat['nH']:.2f}<sup>+{src_stat['nH_u']-src_stat['nH']:.2f}</sup><sub style='position: relative; right: +2.3em;'>-{src_stat['nH']-src_stat['nH_l']:.2}</sub></td>")
    print(f"   <td>{src_stat['nH_LAB']:.2f}</td>")
    print('  </tr>')

  <tr>
   <td>J085927.0-434528</td>
   <td>CV</td>
   <td>2.45</td>
   <td>1.59e-12</td>
   <td>nan</td>
   <td><a class="image-link" href="./images/2CXOJ085927.0-434528_lc.png" target="_blank">1.00</a></td>
   <td>1.0</td>
   <td>6082.5</td>
   <td>1.63<sup>+0.06</sup><sub style='position: relative; right: +2.3em;'>-0.054</sub></td>
   <td><a class="image-link" href="./images/2CXOJ085927.0-434528_pl_spectrum.png" target="_blank">-10.45</a> (<a class="image-link" href="./images/2CXOJ085927.0-434528_pl_corner.pdf" target="_blank">covar</a>)</td>
   <td>7.82<sup>+0.81</sup><sub style='position: relative; right: +2.3em;'>-0.66</sub></td>
   <td><a class="image-link" href="./images/2CXOJ085927.0-434528_mk_spectrum.png" target="_blank">0.00</a> (<a class="image-link" href="./images/2CXOJ085927.0-434528_mk_corner.pdf" target="_blank">covar</a>)</td>
   <td>1.18<sup>+0.02</sup><sub style='position: relative; right: +2.3em;'>-0.021</sub></td>
   <td><a class="image-link" href="./images/2CXOJ08

In [6]:
df_srcs.to_csv('FGL_11152023_GammaCan_summary.csv',index=False)